# Fetch EPA Comment Letters via API

This notebook extracts EPA comment letter records from the E-NEPA API.

Comment letters are included in the main EIS record attachments with `type: "Comment_Letter"`. This notebook:
1. Fetches all EIS records (or uses existing data from `fetch_eis_records_api.ipynb`)
2. Filters to only comment letter attachments
3. Saves a dedicated comment letter metadata file

**API Documentation:** https://cdxapps.epa.gov/cdx-enepa-II/apidocs/index.html

## Output
- `comment_letter_record_api.parquet` - All comment letter records with metadata

In [ ]:
# Install required packages if needed
# !pip install requests pandas pyarrow tqdm

In [ ]:
import requests
import pandas as pd
import json
import time
import os
from datetime import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# Configuration
BASE_URL = "https://cdxapps.epa.gov/cdx-enepa-II/rest"
SEARCH_ENDPOINT = f"{BASE_URL}/public/v1/eis/search"

# Output paths - relative to repository root
REPO_ROOT = Path("../").resolve()
METADATA_DIR = REPO_ROOT / "metadata"

# Ensure metadata directory exists
METADATA_DIR.mkdir(exist_ok=True)

# Input files (from fetch_eis_records_api.ipynb)
EIS_RECORD_PKL = METADATA_DIR / "eis_record_api.pkl"
DOC_RECORD_PKL = METADATA_DIR / "eis_document_record_api.pkl"

# Output files
COMMENT_LETTER_FILE = METADATA_DIR / "comment_letter_record_api.parquet"
COMMENT_LETTER_PKL = METADATA_DIR / "comment_letter_record_api.pkl"

# Year range for fetching (if fetching fresh)
START_YEAR = 1987
END_YEAR = datetime.now().year

# Rate limiting
REQUEST_DELAY = 0.5

print(f"Repository root: {REPO_ROOT}")
print(f"Metadata directory: {METADATA_DIR}")

## Configuration

In [ ]:
# ============================================
# FETCH SETTINGS - MODIFY AS NEEDED
# ============================================

# Set to True to fetch fresh data from the API
# Set to False to use existing data from fetch_eis_records_api.ipynb (faster)
FETCH_FRESH = False

# If FETCH_FRESH=True, set to True to re-fetch ALL records
# Set to False to only fetch NEW records not already in the database
OVERWRITE = False

# Year range (only used if FETCH_FRESH=True)
FETCH_START_YEAR = 1987
FETCH_END_YEAR = datetime.now().year

print(f"=== Fetch Configuration ===")
print(f"  FETCH_FRESH: {FETCH_FRESH}")
if FETCH_FRESH:
    print(f"  OVERWRITE: {OVERWRITE}")
    print(f"  Year range: {FETCH_START_YEAR} to {FETCH_END_YEAR}")
else:
    print(f"  Will use existing data from: {DOC_RECORD_PKL}")

## API Helper Functions

In [ ]:
def search_eis_with_comment_letters(start_date: str, end_date: str, max_retries: int = 3) -> list:
    """
    Search for EIS records that have comment letters within a date range.
    
    Args:
        start_date: Start date in MM/dd/yyyy format
        end_date: End date in MM/dd/yyyy format
        max_retries: Number of retry attempts on failure
    
    Returns:
        List of EIS record dictionaries
    """
    params = {
        "startFRDate": start_date,
        "endFRDate": end_date,
        "onlyCommentLetters": "true"  # Only get records with comment letters
    }
    
    for attempt in range(max_retries):
        try:
            response = requests.get(SEARCH_ENDPOINT, params=params, timeout=60)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            logger.warning(f"Attempt {attempt + 1}/{max_retries} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                logger.error(f"Failed to fetch records for {start_date} to {end_date}")
                return []
    return []


def extract_comment_letters(record: dict) -> list:
    """
    Extract comment letter attachments from an EIS record.
    
    Args:
        record: Raw EIS record from API
    
    Returns:
        List of comment letter dictionaries
    """
    eis_id = record.get("eisId")
    title = record.get("title")
    lead_agency = record.get("leadAgency")
    comment_letter_date = record.get("commentLetterDate")
    attachments = record.get("attachments", [])
    
    # Extract states
    states = record.get("states", [])
    states_str = ", ".join([s.get("name", "") for s in states]) if states else None
    
    letters = []
    for att in attachments:
        # Filter to only comment letters
        if att.get("type") == "Comment_Letter":
            letter = {
                "eisId": eis_id,
                "eisTitle": title,
                "leadAgency": lead_agency,
                "states": states_str,
                "commentLetterDate": comment_letter_date,
                "attachmentId": att.get("id"),
                "name": att.get("name"),
                "title": att.get("title"),
                "fileNameForDownload": att.get("fileNameForDownload"),
                "size": att.get("size"),
                "sizeKb": att.get("sizeKb"),
                "pages": att.get("pages"),
            }
            letters.append(letter)
    
    return letters

## Fetch Comment Letters

In [ ]:
def load_existing_comment_letters():
    """
    Load existing comment letter records if available.
    """
    if COMMENT_LETTER_PKL.exists():
        try:
            df = pd.read_pickle(COMMENT_LETTER_PKL)
            logger.info(f"Loaded {len(df)} existing comment letter records")
            return df
        except Exception as e:
            logger.warning(f"Could not load existing records: {e}")
    return None


def save_comment_letters(df: pd.DataFrame):
    """
    Save comment letter records.
    """
    df.to_pickle(COMMENT_LETTER_PKL)
    df.to_parquet(COMMENT_LETTER_FILE, index=False)
    df.to_csv(METADATA_DIR / "comment_letter_record_api.csv", index=False)
    logger.info(f"Saved {len(df)} comment letter records")


def fetch_from_existing_data():
    """
    Extract comment letters from existing document records.
    """
    if not DOC_RECORD_PKL.exists():
        raise FileNotFoundError(
            f"Document records not found at {DOC_RECORD_PKL}\n"
            "Run fetch_eis_records_api.ipynb first, or set FETCH_FRESH=True"
        )
    
    # Load document records
    doc_df = pd.read_pickle(DOC_RECORD_PKL)
    logger.info(f"Loaded {len(doc_df)} document records")
    
    # Filter to comment letters
    comment_letters = doc_df[doc_df['type'] == 'Comment_Letter'].copy()
    logger.info(f"Found {len(comment_letters)} comment letters")
    
    # Load EIS records to get additional metadata
    if EIS_RECORD_PKL.exists():
        eis_df = pd.read_pickle(EIS_RECORD_PKL)
        # Merge to get EIS title, agency, etc.
        comment_letters = comment_letters.merge(
            eis_df[['eisId', 'title', 'leadAgency', 'states', 'commentLetterDate']],
            on='eisId',
            how='left',
            suffixes=('', '_eis')
        )
        # Rename columns to match fresh fetch format
        comment_letters = comment_letters.rename(columns={'title_eis': 'eisTitle'})
        if 'title' in comment_letters.columns and 'eisTitle' not in comment_letters.columns:
            # title from doc_df is the attachment title, not EIS title
            pass
    
    return comment_letters


def fetch_comment_letters_fresh(start_year: int, end_year: int, overwrite: bool = False):
    """
    Fetch comment letters directly from the API.
    
    Uses the onlyCommentLetters=true filter for efficiency.
    """
    existing_df = load_existing_comment_letters()
    
    if overwrite or existing_df is None:
        all_letters = []
        existing_ids = set()
    else:
        all_letters = existing_df.to_dict('records')
        existing_ids = set(existing_df['attachmentId'].astype(str))
        logger.info(f"Starting with {len(existing_ids)} existing records")
    
    years = list(range(end_year, start_year - 1, -1))
    
    for year in tqdm(years, desc="Fetching years"):
        start_date = f"01/01/{year}"
        end_date = f"12/31/{year}"
        
        records = search_eis_with_comment_letters(start_date, end_date)
        
        new_count = 0
        for record in records:
            letters = extract_comment_letters(record)
            for letter in letters:
                att_id = str(letter.get("attachmentId"))
                if not overwrite and att_id in existing_ids:
                    continue
                all_letters.append(letter)
                existing_ids.add(att_id)
                new_count += 1
        
        logger.info(f"Year {year}: Found {len(records)} EIS with letters, {new_count} new letters")
        time.sleep(REQUEST_DELAY)
    
    return pd.DataFrame(all_letters)

In [ ]:
# Fetch comment letters
if FETCH_FRESH:
    print("Fetching comment letters directly from API...")
    comment_df = fetch_comment_letters_fresh(
        start_year=FETCH_START_YEAR,
        end_year=FETCH_END_YEAR,
        overwrite=OVERWRITE
    )
else:
    print("Extracting comment letters from existing data...")
    comment_df = fetch_from_existing_data()

# Save
save_comment_letters(comment_df)

In [ ]:
# Display summary
print(f"\n=== Comment Letter Summary ===")
print(f"Total comment letters: {len(comment_df)}")

# By year (from EIS ID)
comment_df['year'] = comment_df['eisId'].astype(str).str[:4]
print(f"\nComment letters by year:")
print(comment_df['year'].value_counts().sort_index())

In [ ]:
# Preview records
print("\n=== Sample Records ===")
display_cols = ['eisId', 'attachmentId', 'name', 'sizeKb', 'pages']
if 'eisTitle' in comment_df.columns:
    display_cols.insert(1, 'eisTitle')
if 'leadAgency' in comment_df.columns:
    display_cols.insert(2, 'leadAgency')
display(comment_df[display_cols].head(10))

In [ ]:
# Statistics
print(f"\n=== Statistics ===")
print(f"Total file size: {comment_df['size'].sum() / (1024**3):.2f} GB")
print(f"Average pages per letter: {comment_df['pages'].mean():.1f}")
print(f"Unique EIS projects with letters: {comment_df['eisId'].nunique()}")